In [1]:
import os
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split, Subset
from torchvision import transforms, datasets
from matplotlib import pyplot as plt
import copy
import torch.optim as optim


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\paull\Documents\CODE\MuffinOrChihuahuaCNN\.venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\paull\Documents\CODE\MuffinOrChihuahuaCNN\.venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\paull\Documents\CODE\MuffinOrChihuahuaCNN\.venv\Lib\site-packages\ipykernel\kernelapp

# Import data directly from Kaggle and caches the dataset for further use

In [2]:
dataset_path = '../dataset'

if not os.path.exists(dataset_path) or not os.listdir(dataset_path):
    print("Dataset not found. Downloading now...")
    
    %pip install --quiet kaggle
    
    os.makedirs(dataset_path, exist_ok=True)
    
    !kaggle datasets download -d samuelcortinhas/muffin-vs-chihuahua-image-classification -p {dataset_path}
    
    import zipfile
    with zipfile.ZipFile(os.path.join(dataset_path, 'muffin-vs-chihuahua-image-classification.zip'), 'r') as zip_ref:
        zip_ref.extractall(dataset_path)
    
    os.remove(os.path.join(dataset_path, 'muffin-vs-chihuahua-image-classification.zip'))
    
    print("Dataset downloaded and extracted successfully.")
else:
    print("Dataset already exists. Using cached version.")


Dataset not found. Downloading now...
Note: you may need to restart the kernel to use updated packages.



  0%|          | 0.00/474M [00:00<?, ?B/s]
  0%|          | 1.00M/474M [00:00<01:50, 4.51MB/s]
  1%|          | 3.00M/474M [00:00<00:47, 10.3MB/s]
  2%|▏         | 8.00M/474M [00:00<00:20, 23.9MB/s]
  2%|▏         | 11.0M/474M [00:00<00:26, 18.4MB/s]
  3%|▎         | 14.0M/474M [00:00<00:22, 21.1MB/s]
  4%|▍         | 19.0M/474M [00:00<00:17, 27.0MB/s]
  5%|▌         | 24.0M/474M [00:01<00:14, 31.6MB/s]
  6%|▌         | 29.0M/474M [00:01<00:12, 36.1MB/s]
  7%|▋         | 33.0M/474M [00:01<00:12, 37.2MB/s]
  8%|▊         | 39.0M/474M [00:01<00:10, 43.3MB/s]
  9%|▉         | 44.0M/474M [00:01<00:13, 33.8MB/s]
 11%|█         | 50.0M/474M [00:01<00:11, 40.1MB/s]
 12%|█▏        | 55.0M/474M [00:01<00:10, 40.0MB/s]
 13%|█▎        | 61.0M/474M [00:01<00:09, 45.2MB/s]
 14%|█▍        | 66.0M/474M [00:02<00:09, 43.4MB/s]
 15%|█▍        | 71.0M/474M [00:02<00:09, 45.6MB/s]
 16%|█▌        | 76.0M/474M [00:02<00:09, 46.2MB/s]
 17%|█▋        | 81.0M/474M [00:02<00:08, 46.6MB/s]
 18%|█▊        | 86.

Dataset URL: https://www.kaggle.com/datasets/samuelcortinhas/muffin-vs-chihuahua-image-classification
License(s): CC0-1.0

Dataset downloaded and extracted successfully.


# Loading the test, train and validation sets 

In [106]:
# Parameter to enable development mode
DEV = True  # Set to True for development mode (200 images), False for full dataset

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images
    transforms.ToTensor(),  # Convert images to PyTorch tensors
])

# Define directories
train_dir = '../dataset/train'
test_dir = '../dataset/test'

# Load datasets
full_train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

if DEV:
    # Get all labels from the dataset
    labels = np.array(full_train_dataset.targets)

    # Use NumPy to get indices for each class (Chihuahua and Muffin)
    chihuahua_indices = np.where(labels == 0)[0]  # Class 0 is Chihuahua
    muffin_indices = np.where(labels == 1)[0]     # Class 1 is Muffin

    # Shuffle the indices
    np.random.seed(0)
    np.random.shuffle(chihuahua_indices)
    np.random.shuffle(muffin_indices)

    # Select an equal number of images from each class (100 Chihuahuas, 100 Muffins)
    selected_chihuahua_indices = chihuahua_indices[:100]
    selected_muffin_indices = muffin_indices[:100]

    # Combine the selected indices and shuffle again
    selected_indices = np.concatenate([selected_chihuahua_indices, selected_muffin_indices])
    np.random.shuffle(selected_indices)

    # Split the selected indices into training (160) and validation (40)
    train_indices = selected_indices[:160]
    valid_indices = selected_indices[160:200]

    # Create subsets
    train_dataset = Subset(full_train_dataset, train_indices)
    valid_dataset = Subset(full_train_dataset, valid_indices)
    
    # Similarly, create a subset of the test dataset (first 200 images for testing)
    test_dataset = Subset(test_dataset, range(200))
    
else:
    # Split the training dataset into training and validation sets
    n_train_examples = int(len(full_train_dataset) * 0.8)
    n_valid_examples = len(full_train_dataset) - n_train_examples
    train_dataset, valid_dataset = random_split(full_train_dataset, [n_train_examples, n_valid_examples], generator=torch.manual_seed(0))

# Create the dataloaders
batch_size = 8
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, generator=torch.manual_seed(0))
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)  # Shuffle valid data as well
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

# Print dataset information
print(f'Number of training samples: {len(train_dataset)}')
print(f'Number of validation samples: {len(valid_dataset)}')
print(f'Number of test samples: {len(test_dataset)}')

Number of training samples: 160
Number of validation samples: 40
Number of test samples: 200


# new way

## Init model parameters

In [2]:
def init_params(m, seed=0):
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
        nn.init.xavier_uniform_(m.weight.data, generator=torch.manual_seed(seed))
        if m.bias is not None:
            m.bias.data.fill_(0.01)
    elif isinstance(m, nn.BatchNorm2d):
        nn.init.normal_(m.weight.data, mean=1, std=0.02, generator=torch.manual_seed(seed))
        m.bias.data.fill_(0.01)
    return

## Load and split data

In [3]:
# Chemins des répertoires train et test
train_dir = '../dataset/train'
test_dir = '../dataset/test'

transform = transforms.Compose([
    transforms.Resize((224, 224)),                          # Resize les images à 224x224 pixels
    transforms.ToTensor(),                                  # Convertir l'image en tenseur
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalisation
])

# Charger le dataset d'entraînement complet
train_data = datasets.ImageFolder(root=train_dir, transform=transform)

# Définir les proportions pour l'entraînement et la validation
train_size = int(0.8 * len(train_data))  # 80% pour l'entraînement
valid_size = len(train_data) - train_size  # 20% pour la validation

# Diviser le dataset d'entraînement en entraînement et validation
train_data, valid_data = random_split(train_data, [train_size, valid_size])

# Charger le dataset de test
test_data = datasets.ImageFolder(root=test_dir, transform=transform)

# Définir la taille des batches
batch_size = 32

# Créer les DataLoaders
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# Vérification des tailles
print(f"Taille du train_loader: {len(train_loader.dataset)} images")
print(f"Taille du valid_loader: {len(valid_loader.dataset)} images")
print(f"Taille du test_loader: {len(test_loader.dataset)} images")


Taille du train_loader: 3786 images
Taille du valid_loader: 947 images
Taille du test_loader: 1184 images


## Display a few samples for testing

In [4]:
image_batch_example, labels_batch_example = next(iter(train_loader))

plt.figure(figsize=(10, 6))
for ib in range(batch_size):
    plt.subplot(batch_size // 4, 4, ib + 1)
    
    image = image_batch_example[ib].permute(1, 2, 0).detach().numpy()

    plt.imshow(image)
    plt.xticks([]), plt.yticks([])
    plt.title('Image label = ' + str(labels_batch_example[ib].item()))
    
plt.show()

RuntimeError: Numpy is not available

## Model

In [5]:
class CNNClassif(nn.Module):
    def __init__(self, input_size_linear, num_channels1=16, num_channels2=32, num_classes=2):
        super().__init__()
        self.input = nn.Sequential(nn.Conv2d(3, num_channels1, kernel_size=5, padding=2),nn.ReLU(), nn.MaxPool2d(kernel_size=2))
        self.hidden = nn.Sequential(nn.Conv2d(num_channels1, num_channels2, kernel_size=5, padding=2), nn.ReLU(), nn.MaxPool2d(kernel_size=2))
        self.output = nn.Sequential(nn.Linear(input_size_linear, num_classes), nn.ReLU())
        pass 
       
    def forward(self, x):
        out1 = self.input(x)
        out2 = self.hidden(out1)
        out3 = self.output(out2.reshape(out2.shape[0], -1))
        return out3

In [6]:
# Instantiate the model
num_channels1 = 16
num_channels2 = 32
num_classes = 2
input_size_linear = num_channels2 * 56 * 56
model = CNNClassif(input_size_linear, num_channels1, num_channels2, num_classes)

# Print the total number of parameters in the model
print('Total number of parameters: ', sum(p.numel() for p in model.parameters()))

# initialization of the network's parameters
model.apply(init_params)

Total number of parameters:  214754


CNNClassif(
  (input): Sequential(
    (0): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (hidden): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (output): Sequential(
    (0): Linear(in_features=100352, out_features=2, bias=True)
    (1): ReLU()
  )
)

#### last idea

class CNNClassif(nn.Module):
    def __init__(self, num_channels1=16, num_channels2=32, num_channels3=64, num_classes=2):
        super().__init__()
        
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(3, num_channels1, kernel_size=3, padding=1),  
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  
        )
        
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(num_channels1, num_channels2, kernel_size=3, padding=1),  
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  
        )
        
        self.conv_block3 = nn.Sequential(
            nn.Conv2d(num_channels2, num_channels3, kernel_size=3, padding=1),  
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  
        )
        
        self.flatten = nn.Flatten()

        self._initialize_input_size()
        
        self.fc1 = nn.Linear(self.input_size_linear, 128)  
        self.fc2 = nn.Linear(128, num_classes)  

    def _initialize_input_size(self):
        with torch.no_grad():
            dummy_input = torch.randn(1, 3, 224, 224)  
            out = self.conv_block1(dummy_input)
            out = self.conv_block2(out)
            out = self.conv_block3(out)
            self.input_size_linear = out.view(-1).size(0)

    def forward(self, x):
        out1 = self.conv_block1(x)
        out2 = self.conv_block2(out1)
        out3 = self.conv_block3(out2)
        
        out_flat = self.flatten(out3)
        
        out_fc1 = F.relu(self.fc1(out_flat))
        out_fc2 = self.fc2(out_fc1)
        
        output = F.sigmoid(out_fc2)
        
        return output

## train function

In [7]:
def eval_cnn_classifier(model, eval_dataloader):
    # Set the model in 'evaluation' mode (this disables some layers (batch norm, dropout...) which are not needed when testing)
    model.eval() 

    # In evaluation phase, we don't need to compute gradients (for memory efficiency)
    with torch.no_grad():
        # initialize the total and correct number of labels to compute the accuracy
        correct_labels = 0
        total_labels = 0
        
        # Iterate over the dataset using the dataloader
        for images, labels in eval_dataloader:

            # Get the predicted labels
            #images = images.reshape(images.shape[0], -1)
            y_predicted = model(images)
            
            # To get the predicted labels, we need to get the max over all possible classes
            _, label_predicted = torch.max(y_predicted.data, 1)
            
            # Compute accuracy: count the total number of samples, and the correct labels (compare the true and predicted labels)
            total_labels += labels.size(0)
            correct_labels += (label_predicted == labels).sum().item()
    
    accuracy = 100 * correct_labels / total_labels
    
    return accuracy

def train_val_cnn_classifier(model, train_dataloader, valid_dataloader, num_epochs, loss_fn, learning_rate, verbose=True, seed=0):
    # Make a copy of the model (avoid changing the model outside this function)
    model_tr = copy.deepcopy(model)
    
    # Set the model in 'training' mode (ensures all parameters' gradients are computed - it's like setting 'requires_grad=True' for all parameters)
    model_tr.train()
    
    # Define the optimizer
    optimizer = torch.optim.Adam(model_tr.parameters(), lr=learning_rate)
    
    # Initialize a list for storing the training loss over epochs
    train_losses = []

    best_model = (copy.deepcopy(model_tr), 0)
    val_accuracy = []
    
    # Training loop
    for epoch in range(num_epochs):
        
        # Initialize the training loss for the current epoch
        tr_loss = 0
        
        # Iterate over batches using the dataloader
        for batch_index, (images, labels) in enumerate(train_dataloader):
            #img_vectorize = images.view(images.shape[0], 1*28*28)
            y_pred = model_tr.forward(images)
            loss = loss_fn(y_pred, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step() 
           
            # For each batch, we update the current epoch loss
            tr_loss += loss.item()  

        # At the end of each epoch, get the average training loss over all batches
        tr_loss = tr_loss / len(train_dataloader)
        train_losses.append(tr_loss)

        accuracy = eval_cnn_classifier(model_tr,valid_dataloader)
        if accuracy > best_model[1]:
            best_model = (copy.deepcopy(model_tr), accuracy)
            
        val_accuracy.append(accuracy)
        
        # Display the training loss
        if verbose:
            print('Epoch [{}/{}], Training loss: {:.4f}, Accuracy : {:.4f}'.format(epoch+1, num_epochs, tr_loss, accuracy))

    
    return best_model, train_losses, val_accuracy

In [8]:
# Optimizer
num_epochs = 10
loss_fn = nn.CrossEntropyLoss()
learning_rate = 0.001

# Training
model_tr, train_losses, val_accuracies = train_val_cnn_classifier(model, train_loader, valid_loader, num_epochs, loss_fn, learning_rate, verbose=True)

# Display the training loss and validation accuracy over epochs
plt.figure()
plt.subplot(1, 2, 1)
plt.plot(torch.arange(num_epochs)+1, train_losses)
plt.title('Training loss')
plt.xlabel('Epochs')
plt.subplot(1, 2, 2)
plt.plot(torch.arange(num_epochs)+1, val_accuracies)
plt.title('Validation accuracy')
plt.xlabel('Epochs')
plt.show()

RuntimeError: Numpy is not available